## Description:
这个笔记本， 尝试用协同过滤算法进行新闻的召回， 主要包括用户协同过滤和物品的协同过滤

In [2]:
import gc
import os
import math
#import toad
import pickle
import time
import random

# swifter库，以最快的可用方式将任何函数应用到pandas的DataFrame或者Series用于加速
# 特别适用于一行一行处理DataFrame数据的时候，可以用多个处理器并行处理
import swifter

import numpy as np 
import pandas as pd
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import collections
from utils import reduce_mem, metrics_recall

from CFModel import UserCF, ItemCF

import warnings
warnings.filterwarnings("ignore")

## 导入数据

In [2]:
data_path = 'data_process'
data = pd.read_csv(os.path.join(data_path, 'train_data.csv'), index_col=0, parse_dates=['expo_time'])
# 内存优化下
data = reduce_mem(data)

开始压缩内存...


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 59.37it/s]

压缩前: 571.13 Mb 
压缩后: 435.87 Mb 
压缩比例: (23.68 %)


In [3]:
# 选择出需要用到的列
use_cols = ['user_id', 'article_id', 'expo_time', 'net_status', 'exop_position', 'duration', 'click']
data_new = data[use_cols]

## 划分训练集和测试集
* 训练集， 每个用户的历史点击，去掉最后一次
* 测试集， 每个用户的最后一次点击

In [4]:
# 按照用户分组，然后把最后一个item拿出来
click_df = data_new[data_new['click']==1]

In [5]:
def get_hist_and_last_click(all_click):
    all_click = all_click.sort_values(by=['user_id', 'expo_time'])
    click_last_df = all_click.groupby('user_id').tail(1)
    
    # 如果用户只有一个点击，hist为空了，会导致训练的时候这个用户不可见，此时默认泄露一下
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]

    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)

    return click_hist_df, click_last_df

In [6]:
user_click_hist_df, user_click_last_df = get_hist_and_last_click(click_df)

In [7]:
user_click_hist_df.head()

,user_id,article_id,expo_time,net_status,exop_position,duration,click
0,17340,464481478,2021-06-30 20:34:47,2,21,27,1
1,17340,465148736,2021-07-02 19:35:03,5,23,49,1
2,17340,464707540,2021-07-02 19:47:06,5,25,174,1
3,17340,464993414,2021-07-02 19:47:06,5,27,11,1
4,17340,465115022,2021-07-02 20:01:34,5,41,14,1


## 协同过滤召回
这里采用两种协同过滤， 基于用户的协同过滤和基于物品的协同过滤， 对于每一种协同过滤， 还可以尝试采用关联规则的方式进行优化， 主要是体现在相似分数的计算上

### 用户协同过滤

In [11]:
usercf = UserCF(user_click_hist_df)
user_recall_items_dict = usercf.usercf_recommend()

计算用户相似性矩阵.....
用户协同过滤召回....


 16%|████████████▏                                                                | 3154/20000 [05:42<22:50, 12.29it/s]

召回数量不足, 这里随机从看过的文章里面选....


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [33:00<00:00, 10.10it/s]


In [11]:
usercf_corr = UserCF(user_click_hist_df, sim_corr_rule=True)
user_recall_items_dict_corr = usercf_corr.usercf_recommend()

计算用户相似性矩阵.....


100%|███████████████████████████████████████████████████████████████████████████| 47533/47533 [06:55<00:00, 114.44it/s]


用户协同过滤召回....


 16%|████████████▏                                                                | 3154/20000 [13:29<48:16,  5.82it/s]

召回数量不足, 这里随机从看过的文章里面选....


100%|██████████████████████████████████████████████████████████████████████████| 20000/20000 [1:20:41<00:00,  4.13it/s]


### 文章协同过滤

In [16]:
itemcf = ItemCF(user_click_hist_df)
user_recall_items_dict_itemcf = itemcf.itemcf_recommend()

计算文章相似性矩阵.....
文章协同过滤召回....


 12%|█████████▎                                                                   | 2425/20000 [01:26<10:32, 27.80it/s]

召回数量不足, 这里随机从看过的文章里面选....


 16%|████████████▏                                                                | 3153/20000 [01:50<08:56, 31.43it/s]

召回数量不足, 这里随机从看过的文章里面选....


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [11:30<00:00, 28.94it/s]


In [17]:
itemcf_corr = ItemCF(user_click_hist_df, sim_corr_rule=True)
user_recall_items_dict_itemcf_corr = itemcf_corr.itemcf_recommend()

计算文章相似性矩阵.....


100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [02:54<00:00, 114.60it/s]


文章协同过滤召回....


 12%|█████████▎                                                                   | 2424/20000 [02:01<13:55, 21.05it/s]

召回数量不足, 这里随机从看过的文章里面选....


 16%|████████████▏                                                                | 3154/20000 [02:37<14:41, 19.11it/s]

召回数量不足, 这里随机从看过的文章里面选....


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [16:15<00:00, 20.51it/s]


## 召回评估

In [73]:
# 评估不加关联规则的用户协同过滤召回
metrics_recall(user_recall_items_dict, user_click_last_df, topk=200)

 topk:  50  :  hit_num:  2033 hit_rate:  0.10165 user_num :  20000
 topk:  100  :  hit_num:  3033 hit_rate:  0.15165 user_num :  20000
 topk:  150  :  hit_num:  3657 hit_rate:  0.18285 user_num :  20000
 topk:  200  :  hit_num:  4121 hit_rate:  0.20605 user_num :  20000


In [12]:
# 评估加关联规则的用户协同过滤召回
metrics_recall(user_recall_items_dict_corr, user_click_last_df, topk=200)

 topk:  50  :  hit_num:  2022 hit_rate:  0.1011 user_num :  20000
 topk:  100  :  hit_num:  3016 hit_rate:  0.1508 user_num :  20000
 topk:  150  :  hit_num:  3763 hit_rate:  0.18815 user_num :  20000
 topk:  200  :  hit_num:  4339 hit_rate:  0.21695 user_num :  20000


In [18]:
# 评估不加关联规则的文章协同过滤召回
metrics_recall(user_recall_items_dict_itemcf, user_click_last_df, topk=200)

 topk:  50  :  hit_num:  2055 hit_rate:  0.10275 user_num :  20000
 topk:  100  :  hit_num:  2956 hit_rate:  0.1478 user_num :  20000
 topk:  150  :  hit_num:  3555 hit_rate:  0.17775 user_num :  20000
 topk:  200  :  hit_num:  4040 hit_rate:  0.202 user_num :  20000


In [18]:
# 评估加关联规则的文章协同过滤召回
metrics_recall(user_recall_items_dict_itemcf_corr , user_click_last_df, topk=200)

 topk:  50  :  hit_num:  2185 hit_rate:  0.10925 user_num :  20000
 topk:  100  :  hit_num:  3142 hit_rate:  0.1571 user_num :  20000
 topk:  150  :  hit_num:  3828 hit_rate:  0.1914 user_num :  20000
 topk:  200  :  hit_num:  4347 hit_rate:  0.21735 user_num :  20000
